In [1]:
from nba_api.stats.endpoints import ShotChartDetail, LeagueGameLog, BoxScorePlayerTrackV3, BoxScoreAdvancedV3, playbyplayv3, SynergyPlayTypes, BoxScoreTraditionalV3
import pandas as pd
import time
#seasons = ['2020-21','2021-22','2022-23','2023-24']
#seasons = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20','2020-21','2021-22','2022-23','2023-24']
seasons = ['2024-25']

# Data Fetching
Data gathered through the NBA API with the basic pattern for most endpoints being get the games in a season and query the endpoint for each game. Play type data is not done on a game by game basis so is retrived sepratly. 

In [2]:
def get_game_ids(season, season_type='Regular Season'):
    game_log = LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
    game_ids = game_log['GAME_ID'].unique().tolist()
    return game_ids

def get_data(game_ids):
    play_by_play_data = []
    shotchart_data = []
    playertracking_data = []
    advanced_data = []
    box_data = []
    
    for game_id in game_ids:
        try:
            #standard box
            try:
                box = BoxScoreTraditionalV3(game_id = game_id).get_data_frames()[0]
                box['GAME_ID'] = game_id
                box_data.append(box)
            except Exception as e:
                print(f"Failed box traditional for {game_id}: {e}")
                time.sleep(1)
            
            #play by play
            try:
                play_by_play = playbyplayv3.PlayByPlayV3(game_id=game_id).get_data_frames()[0]
                play_by_play['GAME_ID'] = game_id
                play_by_play_data.append(play_by_play)
            except Exception as e:
                print(f"Failed PlayByPlayV2 for {game_id}: {e}")
                time.sleep(1)
                
            #ShotChartDetail
            try:
                shotchart = ShotChartDetail(game_id_nullable=game_id, team_id=0, player_id=0,context_measure_simple='FGA').get_data_frames()[0]
                shotchart['GAME_ID'] = game_id
                shotchart_data.append(shotchart)
            except Exception as e:
                print(f"Failed ShotChartDetail for {game_id}: {e}")
                time.sleep(1) 
            
            #BoxScorePlayerTrackV3
            try:
                player_tracking = BoxScorePlayerTrackV3(game_id=game_id).get_data_frames()[0]
                player_tracking['GAME_ID'] = game_id
                playertracking_data.append(player_tracking)
            except Exception as e:
                print(f"Failed BoxScorePlayerTrackV3 for {game_id}: {e}")
                time.sleep(1)
            
            #BoxScoreAdvancedV3
            try:
                advanced_boxscore = BoxScoreAdvancedV3(game_id=game_id).get_data_frames()[0]
                advanced_boxscore['GAME_ID'] = game_id
                advanced_data.append(advanced_boxscore)
            except Exception as e:
                print(f"Failed BoxScoreAdvancedV3 for {game_id}: {e}")
                time.sleep(1)
            
            # rate limiting
            time.sleep(0.6)
        
        except Exception as e:
            print(f"Unexpected error for game {game_id}: {e}")
            time.sleep(10) 
    
    # Concat dataframes only if data exists
    shotchart_df = pd.concat(shotchart_data, ignore_index=True) if shotchart_data else pd.DataFrame()
    playertracking_df = pd.concat(playertracking_data, ignore_index=True) if playertracking_data else pd.DataFrame()
    advanced_df = pd.concat(advanced_data, ignore_index=True) if advanced_data else pd.DataFrame()
    play_by_play_df = pd.concat(play_by_play_data, ignore_index=True) if play_by_play_data else pd.DataFrame()
    traditional_box_df = pd.concat(box_data,ignore_index=True) if box_data else pd.DataFrame()
    
    return shotchart_df, playertracking_df, advanced_df,play_by_play_df, traditional_box_df

for season in seasons:
    game_ids = get_game_ids(season = season)


    shotchart_df, playertracking_df,advanced_box_df,play_by_play_df,traditional_box_df = get_data(game_ids)
    
    #names
    shotchart_file = f"{season}_shotchart_data.csv"
    playertracking_file = f"{season}_playertracking_data.csv"
    advanced_box_file = f"{season}_advanced_box_data.csv"
    play_by_play_file = f'{season}_play_by_play.csv'
    traditional_boxfile = f"{season}_box_data.csv"

    
    shotchart_df.to_csv(shotchart_file, index=False)
    playertracking_df.to_csv(playertracking_file, index=False)
    advanced_box_df.to_csv(advanced_box_file,index=False)
    play_by_play_df.to_csv(play_by_play_file, index=False)
    traditional_box_df.to_csv(traditional_boxfile, index=False)


In [7]:
league_id = '00'  # NBA
season_type = 'Regular Season'
per_mode = 'PerGame'
player_or_team = 'P'
play_types = ['Cut', 'Handoff', 'Isolation', 'Misc', 'OffScreen', 'Postup', 'PRBallHandler', 'PRRollman', 'OffRebound', 'Spotup', 'Transition']
type_groupings = ['offensive', 'defensive']

for season in seasons:
    synergy_data = [] 
    
    for type_grouping in type_groupings:
        for play_type in play_types:
            try:
                # Fetch the synergy data
                synergy = SynergyPlayTypes(
                    league_id=league_id,
                    per_mode_simple=per_mode,
                    player_or_team_abbreviation=player_or_team,
                    season_type_all_star=season_type,
                    season=season,
                    play_type_nullable=play_type,
                    type_grouping_nullable=type_grouping
                ).get_data_frames()[0]
                
                synergy_data.append(synergy)
                time.sleep(0.6)  # Delay for rate limiting

            except Exception as e:
                print(f"Failed Synergy for season: {season}, play type: {play_type}, type grouping: {type_grouping}: {e}")
                time.sleep(1)  # Delay before retrying on error

    
    synergy_df = pd.concat(synergy_data, ignore_index=True) if synergy_data else pd.DataFrame()

    # Save
    play_type_file = f'{season}_play_type.csv'
    synergy_df.to_csv(play_type_file, index=False)




        

/tmp/ipykernel_506292/233088658.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  synergy_df = pd.concat(synergy_data, ignore_index=True) if synergy_data else pd.DataFrame()
/tmp/ipykernel_506292/233088658.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  synergy_df = pd.concat(synergy_data, ignore_index=True) if synergy_data else pd.DataFrame()
/tmp/ipykernel_506292/233088658.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a fut

In [12]:
import re

def parse_failed_ids(asdf):
    failed_ids = {
        "BoxScoreTraditional": [],
        "PlayByPlay": [],
        "ShotChartDetail": [],
        "BoxScorePlayerTrack": [],
        "BoxScoreAdvanced": []
    }
    
    # Define regex patterns for each category
    patterns = {
        "BoxScoreTraditional": r"Failed box traditional for (\d+):",
        "PlayByPlay": r"Failed PlayByPlayV2 for (\d+):",
        "ShotChartDetail": r"Failed ShotChartDetail for (\d+):",
        "BoxScorePlayerTrack": r"Failed BoxScorePlayerTrackV3 for (\d+):",
        "BoxScoreAdvanced": r"Failed BoxScoreAdvancedV3 for (\d+):"
    }
    
    for category, pattern in patterns.items():
        matches = re.findall(pattern, asdf)
        failed_ids[category].extend(matches)
    
    return failed_ids


failed_ids = parse_failed_ids(asdf)
print(failed_ids)


{'BoxScoreTraditional': ['0021500241', '0021700370'], 'PlayByPlay': ['0022100301', '0022100506', '0022200707', '0021500192', '0021500242', '0021700079', '0021700319', '0021700370', '0021800264', '0021900799'], 'ShotChartDetail': ['0021700370', '0021700916', '0021900588'], 'BoxScorePlayerTrack': ['0021700370', '0021700558'], 'BoxScoreAdvanced': ['0021700120']}


In [14]:
#GPT

import pandas as pd
import os
from time import sleep

# Define the fetch functions for each category
fetch_functions = {
    "BoxScoreTraditional": BoxScoreTraditionalV3,
    "PlayByPlay": playbyplayv3.PlayByPlayV3,
    "ShotChartDetail": ShotChartDetail,
    "BoxScorePlayerTrack": BoxScorePlayerTrackV3,
    "BoxScoreAdvanced": BoxScoreAdvancedV3
}

# Determine the season from the game ID
def get_season_from_game_id(game_id):
    year_prefix = game_id[:4]  # Extract the first four characters (e.g., "0022" for 2022)
    return f"{int(year_prefix) + 2000}"  # Adjust to the format (e.g., "2022")

# Function to fetch data with retries
def fetch_data_with_retries(fetch_function, game_id, **kwargs):
    for attempt in range(3):  # Number of retries
        try:
            data = fetch_function(game_id=game_id, **kwargs).get_data_frames()[0]
            return data
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for game {game_id}: {e}")
            sleep(1)  # Delay between attempts
    print(f"Failed after 3 attempts for game {game_id}")
    return pd.DataFrame()

# Update the data files with missing data
def update_data_files(failed_ids):
    for category, game_ids in failed_ids.items():
        file_name = {
            "BoxScoreTraditional": "boxscoretraditional_data.csv",
            "PlayByPlay": "play_by_play.csv",
            "ShotChartDetail": "shotchart_data.csv",
            "BoxScorePlayerTrack": "boxscoreplayertrack_data.csv",
            "BoxScoreAdvanced": "advanced_box_data.csv"
        }.get(category, None)

        if not file_name:
            print(f"Unknown category: {category}")
            continue

        # Determine the season based on the game IDs
        first_game_id = game_ids[0]  # Use the first game ID to determine the season
        season = get_season_from_game_id(first_game_id)
        file_path = f"{season}_{file_name}"

        # Load existing data if the file exists
        existing_df = pd.read_csv(file_path) if os.path.exists(file_path) else pd.DataFrame()

        for game_id in game_ids:
            print(f"Fetching data for {category} game ID {game_id}")
            if category == "BoxScoreTraditional":
                new_data = fetch_data_with_retries(fetch_functions[category], game_id)
            elif category == "PlayByPlay":
                new_data = fetch_data_with_retries(fetch_functions[category], game_id)
            elif category == "ShotChartDetail":
                new_data = fetch_data_with_retries(fetch_functions[category], game_id, team_id=0, player_id=0, context_measure_simple='FGA')
            elif category == "BoxScorePlayerTrack":
                new_data = fetch_data_with_retries(fetch_functions[category], game_id)
            elif category == "BoxScoreAdvanced":
                new_data = fetch_data_with_retries(fetch_functions[category], game_id)
            else:
                print(f"Unknown category: {category}")
                continue

            if not new_data.empty:
                # Append new data to the existing DataFrame
                existing_df = pd.concat([existing_df, new_data], ignore_index=True)
                print(f"Appended data for game ID {game_id}")
            else:
                print(f"No data fetched for game ID {game_id}")

            # Optional: Avoid hitting rate limits
            sleep(1)

        # Save the updated DataFrame back to CSV
        existing_df.to_csv(file_path, index=False)
        print(f"Updated file saved: {file_path}")

update_data_files(failed_ids)


Fetching data for BoxScoreTraditional game ID 0021500241
Appended data for game ID 0021500241
Fetching data for BoxScoreTraditional game ID 0021700370
Appended data for game ID 0021700370
Updated file saved: 2021_boxscoretraditional_data.csv
Fetching data for PlayByPlay game ID 0022100301
Appended data for game ID 0022100301
Fetching data for PlayByPlay game ID 0022100506
Appended data for game ID 0022100506
Fetching data for PlayByPlay game ID 0022200707
Appended data for game ID 0022200707
Fetching data for PlayByPlay game ID 0021500192
Appended data for game ID 0021500192
Fetching data for PlayByPlay game ID 0021500242
Appended data for game ID 0021500242
Fetching data for PlayByPlay game ID 0021700079
Appended data for game ID 0021700079
Fetching data for PlayByPlay game ID 0021700319
Appended data for game ID 0021700319
Fetching data for PlayByPlay game ID 0021700370
Appended data for game ID 0021700370
Fetching data for PlayByPlay game ID 0021800264
Appended data for game ID 0021

In [3]:
import requests
import json
base_url = "https://cdn.nba.com/static/json/staticData/scheduleLeagueV2.json"

response = requests.get(base_url)
if response.status_code == 200:
    base_data = response.json()
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")

schedule_url = base_data.get(base_url)  # Replace 'link' with the actual key that stores the URL.

schedule_response = requests.get(schedule_url)
if schedule_response.status_code == 200:
    schedule_data = schedule_response.json()
else:
    print(f"Failed to fetch schedule. HTTP Status Code: {schedule_response.status_code}")


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

In [12]:
import requests
import pandas as pd
import pandas as pd
from datetime import datetime, timedelta

# URL for NBA Stats API
url = "https://stats.nba.com/stats/scheduleleaguev2"

# Parameters
params = {
    "Season": "2024-25",
    "LeagueID": "00",
}

# Headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.nba.com/",
    "Origin": "https://www.nba.com",
    "Connection": "keep-alive",
}

# Send GET request
response = requests.get(url, headers=headers, params=params)

# Check for successful response
if response.status_code == 200:
    data = response.json()
else:
    print(f"Request failed with status code: {response.status_code}")



In [13]:
# Extract the games data
game_dates = data['leagueSchedule']['gameDates']
all_games = []

for date_info in game_dates:
    game_date = date_info['gameDate']
    games = date_info['games']
    for game in games:
        game['gameDate'] = game_date  # Add the date to each game record
        all_games.append(game)

# Convert to DataFrame
df = pd.DataFrame(all_games)


# Parse gameDate into datetime objects
df['gameDate'] = pd.to_datetime(df['gameDate'], format='%m/%d/%Y %H:%M:%S')

# Define current date and one week ahead
current_date = datetime.utcnow()
next_week = current_date + timedelta(weeks=1)

# Filter games for the next week
next_week_games = df[(df['gameDate'] >= current_date) & (df['gameDate'] <= next_week)]


In [34]:
df.columns

Index(['gameId', 'gameCode', 'gameStatus', 'gameStatusText', 'gameSequence',
       'gameDateEst', 'gameTimeEst', 'gameDateTimeEst', 'gameDateUTC',
       'gameTimeUTC', 'gameDateTimeUTC', 'awayTeamTime', 'homeTeamTime', 'day',
       'monthNum', 'weekNumber', 'weekName', 'ifNecessary', 'seriesGameNumber',
       'gameLabel', 'gameSubLabel', 'seriesText', 'arenaName', 'arenaState',
       'arenaCity', 'postponedStatus', 'branchLink', 'gameSubtype',
       'broadcasters', 'homeTeam', 'awayTeam', 'pointsLeaders', 'gameDate',
       'homeTeamName'],
      dtype='object')

In [37]:

pd.set_option('display.max_colwidth', None)
home = pd.json_normalize(df['homeTeam'])
away = pd.json_normalize(df['awayTeam'])

teams_combined = pd.concat(
    [home[['teamId']].rename(columns={'teamId': 'home_teamId'}),
     away[['teamId']].rename(columns={'teamId': 'away_teamId'})],
    axis=1
)
teams_combined

,home_teamId,away_teamId
0,1610612743,1610612738
1,1610612762,15020
2,1610612747,1610612750
3,1610612746,1610612744
4,1610612738,1610612743
...,...,...
1276,1610612740,1610612760
1277,1610612759,1610612761
1278,1610612744,1610612746
1279,1610612757,1610612747
